In [1]:
import pandas as pd
import numpy as np
from collections import defaultdict
from sklearn.metrics import accuracy_score, precision_score


In [16]:
get_ipython().system('wget -O heart.csv https://archive.ics.uci.edu/ml/machine-learning-databases/heart-disease/processed.cleveland.data')

--2026-01-01 08:29:09--  https://archive.ics.uci.edu/ml/machine-learning-databases/heart-disease/processed.cleveland.data
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified
Saving to: ‘heart.csv’

heart.csv               [ <=>                ]  18.03K  --.-KB/s    in 0.06s   

2026-01-01 08:29:10 (293 KB/s) - ‘heart.csv’ saved [18461]



In [32]:
class ExpertSystem:
    def __init__(self):
        self.rules = []
        self.facts = set()
        self.fired_rules = set()  # refraction

    def add_rule(self, rule):
        self.rules.append(rule)

    def reset(self):
        self.facts.clear()
        self.fired_rules.clear()

In [48]:
def forward_chaining(self):
        fired = True
        while fired:
            fired = False

            applicable_rules = sorted(
                self.rules,
                key=lambda r: r["priority"],
                reverse=True
            )

            for rule in applicable_rules:
                rule_id = id(rule)
                if rule_id in self.fired_rules:
                    continue

                if all(cond in self.facts for cond in rule["conditions"]):
                    self.facts.add(rule["conclusion"])
                    self.fired_rules.add(rule_id)
                    fired = True

In [64]:
def backward_chaining(self, goal):
        if goal in self.facts:
            return True

        for rule in self.rules:
            if rule["conclusion"] == goal:
                if all(self.backward_chaining(cond) for cond in rule["conditions"]):
                    self.facts.add(goal)
                    return True
        return False

In [73]:
def fuzzify_patient(self, age, cholesterol):
        if Fuzzy.high(age, 45, 70) > 0.6:
            self.facts.add("old_age")

        if Fuzzy.high(cholesterol, 200, 300) > 0.6:
            self.facts.add("high_cholesterol")

In [74]:
class Fuzzy:
    @staticmethod
    def low(x, a, b):
        return max(0, min(1, (b - x) / (b - a)))

    @staticmethod
    def medium(x, a, b, c):
        return max(0, min((x - a) / (b - a), (c - x) / (c - b)))

    @staticmethod
    def high(x, b, c):
        return max(0, min(1, (x - b) / (c - b)))

class ExpertSystem:
    def __init__(self):
        self.rules = []
        self.facts = set()
        self.fired_rules = set()  # refraction

    def add_rule(self, rule):
        self.rules.append(rule)

    def reset(self):
        self.facts.clear()
        self.fired_rules.clear()

    def forward_chaining(self):
        fired = True
        while fired:
            fired = False

            applicable_rules = sorted(
                self.rules,
                key=lambda r: r["priority"],
                reverse=True
            )

            for rule in applicable_rules:
                rule_id = id(rule)
                if rule_id in self.fired_rules:
                    continue

                if all(cond in self.facts for cond in rule["conditions"]):
                    self.facts.add(rule["conclusion"])
                    self.fired_rules.add(rule_id)
                    fired = True

    def backward_chaining(self, goal):
        if goal in self.facts:
            return True

        for rule in self.rules:
            if rule["conclusion"] == goal:
                if all(self.backward_chaining(cond) for cond in rule["conditions"]):
                    self.facts.add(goal)
                    return True
        return False

    def fuzzify_patient(self, age, cholesterol):
        if Fuzzy.high(age, 45, 70) > 0.6:
            self.facts.add("old_age")

        if Fuzzy.high(cholesterol, 200, 300) > 0.6:
            self.facts.add("high_cholesterol")

In [83]:
expert = ExpertSystem()

expert.add_rule({
    "conditions": ["high_bp", "high_cholesterol"],
    "conclusion": "heart_disease",
    "priority": 3
})

expert.add_rule({
    "conditions": ["old_age"],
    "conclusion": "high_risk",
    "priority": 2
})

expert.add_rule({
    "conditions": ["high_risk", "heart_disease"],
    "conclusion": "critical_case",
    "priority": 5
})

In [87]:
columns = ['age', 'sex', 'cp', 'trestbps', 'chol', 'fbs', 'restecg', 'thalach', 'exang', 'oldpeak', 'slope', 'ca', 'thal', 'target']

data = pd.read_csv(
    "heart.csv",
    sep=',',  # Corrected separator to comma
    header=None,
    names=columns,
    na_values='?',
    engine='python'
)

# Convert specified columns to numeric, forcing errors to NaN
data['ca'] = pd.to_numeric(data['ca'], errors='coerce')
data['thal'] = pd.to_numeric(data['thal'], errors='coerce')

# Drop rows with any missing values
data.dropna(inplace=True)

# Convert 'target' column to binary (values > 0 become 1, 0 remains 0)
data['target'] = data['target'].apply(lambda x: 1 if x > 0 else 0)

y_true = []
y_pred = []

for _, row in data.iterrows():
    expert.reset()

    if row["trestbps"] > 140:
        expert.facts.add("high_bp")

    expert.fuzzify_patient(row["age"], row["chol"])

    expert.forward_chaining()

    prediction = expert.backward_chaining("heart_disease")

    y_pred.append(int(prediction))
    y_true.append(row["target"])

In [88]:
accuracy = accuracy_score(y_true, y_pred)
print(f"Accuracy: {accuracy:.2f}")

Accuracy: 0.57


In [89]:
precision = precision_score(y_true, y_pred)
print(f"Precision: {precision:.2f}")

Precision: 0.68


## Summary:


### Data Analysis Key Findings
*   The `heart.csv` dataset was successfully loaded, parsing missing values represented by '?', and the `target` column was binarized (values > 0 became 1, 0 remained 0).
*   The `y_true` and `y_pred` lists were populated by running an expert system; for each patient, the system was reset, `high_bp` fact added if `trestbps > 140`, patient data fuzzified using `age` and `chol`, and predictions generated via forward and backward chaining.
*   The accuracy of the expert system's predictions was calculated to be 0.57.
*   The precision of the expert system's predictions was calculated to be 0.68.

### Insights or Next Steps
*   An accuracy of 0.57 suggests the expert system's overall predictive power is slightly better than a random guess for a binary classification task, indicating significant room for improvement.
*   A precision of 0.68 means that when the expert system predicts the presence of heart disease, it is correct 68\% of the time. Depending on the application, this level of precision might be acceptable, but further optimization could reduce false positive predictions.
